In [184]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import importlib
import scraper_utils as su
%load_ext autoreload
%autoreload 2
import time
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data in

Load data in from csvs, convert categories and types columns to lists (saved as strings)

In [254]:
cols = ['Name', 'id', 'baverage', 'average','numwish', 'numowned', 'numplays','numfans', 'avgweight',  'yearpublished',
       'minplayers', 'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'categories', 'types']
df_list = []
for i in range(10):
    fname=f'data/partial_data/df{i}.csv'
    df = pd.read_csv(fname,usecols = cols)
    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df['categories']= df['categories'].apply(literal_eval)
df['types']= df['types'].apply(literal_eval)


In [255]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           10000 non-null  object 
 1   id             10000 non-null  int64  
 2   average        10000 non-null  float64
 3   baverage       10000 non-null  float64
 4   avgweight      10000 non-null  float64
 5   numwish        10000 non-null  float64
 6   numowned       10000 non-null  float64
 7   numplays       10000 non-null  float64
 8   numfans        10000 non-null  float64
 9   yearpublished  10000 non-null  float64
 10  minplayers     10000 non-null  float64
 11  maxplayers     10000 non-null  float64
 12  minplaytime    10000 non-null  float64
 13  maxplaytime    10000 non-null  float64
 14  minage         10000 non-null  float64
 15  categories     10000 non-null  object 
 16  types          10000 non-null  object 
dtypes: float64(13), int64(1), object(3)
memory usage: 1

In [256]:
df

,Name,id,average,baverage,avgweight,numwish,numowned,numplays,numfans,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,categories,types
0,Gloomhaven,174430,8.79923,8.55546,3.8549,15900.0,66360.0,332850.0,7003.0,2017.0,1.0,4.0,60.0,120.0,14.0,"[2689, 2839, 1022, 2018, 2857, 2893, 2023, 300...","[thematic, strategy]"
1,Pandemic Legacy: Season 1,161936,8.61526,8.46581,2.8400,10696.0,64193.0,228424.0,2779.0,2015.0,2.0,4.0,60.0,60.0,13.0,"[2001, 2023, 1084, 2040, 2824, 2145, 2078, 200...","[thematic, strategy]"
2,Brass: Birmingham,224517,8.66068,8.36360,3.9170,9226.0,27062.0,43737.0,1459.0,2018.0,2.0,4.0,60.0,120.0,14.0,"[2883, 1021, 2040, 2902, 1088, 2904, 2900, 208...",[strategy]
3,Terraforming Mars,167791,8.43238,8.28551,3.2388,17193.0,84252.0,349382.0,5550.0,2016.0,1.0,5.0,120.0,120.0,12.0,"[2041, 2984, 1021, 2875, 1084, 2040, 2026, 290...",[strategy]
4,Twilight Imperium: Fourth Edition,233078,8.69556,8.24081,4.2224,7813.0,16328.0,18830.0,1582.0,2017.0,3.0,6.0,240.0,480.0,14.0,"[2838, 2080, 2021, 1015, 2072, 1021, 2843, 267...","[thematic, strategy]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Let's Take a Hike,104063,5.89506,5.54878,1.3500,69.0,328.0,473.0,12.0,2011.0,2.0,5.0,20.0,30.0,5.0,"[2041, 1002, 1041, 2040]",[family]
9996,Doodle China,205790,6.47051,5.54876,2.0000,47.0,129.0,242.0,30.0,2016.0,1.0,5.0,20.0,40.0,8.0,"[1029, 2039]",[]
9997,Web and Starship,2986,6.31991,5.54876,3.5333,86.0,352.0,48.0,12.0,1984.0,2.0,3.0,120.0,120.0,12.0,"[2689, 2963, 2081, 2078, 1016, 1019]",[war]
9998,Slyville,266936,6.85562,5.54876,1.5000,54.0,106.0,129.0,41.0,2019.0,3.0,5.0,45.0,60.0,10.0,"[2012, 1023, 2040, 1079, 2902, 1035, 2011, 102...",[]


## Convert categories from code to names

In [179]:
category_dict = su.make_category_dict()

In [257]:
df['categories']=df['categories'].apply(lambda x: su.code_to_name(category_dict,x))

In [258]:
df

,Name,id,average,baverage,avgweight,numwish,numowned,numplays,numfans,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,categories,types
0,Gloomhaven,174430,8.79923,8.55546,3.8549,15900.0,66360.0,332850.0,7003.0,2017.0,1.0,4.0,60.0,120.0,14.0,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[thematic, strategy]"
1,Pandemic Legacy: Season 1,161936,8.61526,8.46581,2.8400,10696.0,64193.0,228424.0,2779.0,2015.0,2.0,4.0,60.0,60.0,13.0,"[Environmental, Medical]","[thematic, strategy]"
2,Brass: Birmingham,224517,8.66068,8.36360,3.9170,9226.0,27062.0,43737.0,1459.0,2018.0,2.0,4.0,60.0,120.0,14.0,"[Economic, Industry / Manufacturing, Transport...",[strategy]
3,Terraforming Mars,167791,8.43238,8.28551,3.2388,17193.0,84252.0,349382.0,5550.0,2016.0,1.0,5.0,120.0,120.0,12.0,"[Economic, Environmental, Industry / Manufactu...",[strategy]
4,Twilight Imperium: Fourth Edition,233078,8.69556,8.24081,4.2224,7813.0,16328.0,18830.0,1582.0,2017.0,3.0,6.0,240.0,480.0,14.0,"[Civilization, Economic, Negotiation, Politica...","[thematic, strategy]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Let's Take a Hike,104063,5.89506,5.54878,1.3500,69.0,328.0,473.0,12.0,2011.0,2.0,5.0,20.0,30.0,5.0,"[Card Game, Children's Game]",[family]
9996,Doodle China,205790,6.47051,5.54876,2.0000,47.0,129.0,242.0,30.0,2016.0,1.0,5.0,20.0,40.0,8.0,[City Building],[]
9997,Web and Starship,2986,6.31991,5.54876,3.5333,86.0,352.0,48.0,12.0,1984.0,2.0,3.0,120.0,120.0,12.0,"[Science Fiction, Wargame]",[war]
9998,Slyville,266936,6.85562,5.54876,1.5000,54.0,106.0,129.0,41.0,2019.0,3.0,5.0,45.0,60.0,10.0,"[Bluffing, Humor, Medieval, Negotiation]",[]


## Get marketplace data

Iterate through all ids, get marketplace data, keep track of which ids didn't have any for sale (in USD)

In [18]:
all_ids = df['id']

In [52]:
marketplace_list = []
no_listings = []
start = time.time()   
for i,id_ in enumerate(all_ids):
        mp_data = su.get_marketplace_data(id_)
        if mp_data[0]==0: no_listings.append(id_)
        marketplace_list.append(mp_data)
        if i%500 == 0:
            print(i, 'done, last 500 took', str(time.time()-start))
            start = time.time()

0 done, last 500 took 0.1080179214477539
500 done, last 500 took 34.67627716064453
1000 done, last 500 took 31.44352102279663
1500 done, last 500 took 30.87442684173584
2000 done, last 500 took 30.23093605041504
2500 done, last 500 took 29.912564992904663
3000 done, last 500 took 29.44926118850708
3500 done, last 500 took 32.20809197425842
4000 done, last 500 took 28.949753999710083
4500 done, last 500 took 29.182222843170166
5000 done, last 500 took 29.20558500289917
5500 done, last 500 took 68.37382674217224
6000 done, last 500 took 76.51822209358215
6500 done, last 500 took 76.23167276382446
7000 done, last 500 took 78.71208596229553
7500 done, last 500 took 79.2893226146698
8000 done, last 500 took 74.49512815475464
8500 done, last 500 took 74.36403894424438
9000 done, last 500 took 71.50854897499084
9500 done, last 500 took 69.66451907157898


Save as a DataFrame and convert to csv

In [118]:
mp_df = pd.DataFrame(marketplace_list, columns = ['num_listings','median_price'])
mp_df.to_csv('data/partial_data/marketplace_data.csv', index=False)


## Combine marketplace data and other stats into 1 df

Also do cleaning - get rid of things with low owned and num plays values, and with no marketplace data. Also filter by games that have come out since 2010 - this is what actually does more of the filtering. This leaves us with less than half the data we started with. That's ok because we still have almost 5000 entries and most of what we got rid of is old.

In [259]:
df_with_mp = pd.concat([df,mp_df], axis=1)
cleaning_mask = (df_with_mp['numowned'] > 100)&\
                (df_with_mp['numplays'] > 100)&\
                (df_with_mp['yearpublished'] >= 2010)&\
                (df_with_mp['types'].apply(lambda x: len(x)) > 0)&\
                (df_with_mp['num_listings'] > 0)
df_cleaned=df_with_mp[cleaning_mask].copy()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2823 entries, 0 to 9995
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           2823 non-null   object 
 1   id             2823 non-null   int64  
 2   average        2823 non-null   float64
 3   baverage       2823 non-null   float64
 4   avgweight      2823 non-null   float64
 5   numwish        2823 non-null   float64
 6   numowned       2823 non-null   float64
 7   numplays       2823 non-null   float64
 8   numfans        2823 non-null   float64
 9   yearpublished  2823 non-null   float64
 10  minplayers     2823 non-null   float64
 11  maxplayers     2823 non-null   float64
 12  minplaytime    2823 non-null   float64
 13  maxplaytime    2823 non-null   float64
 14  minage         2823 non-null   float64
 15  categories     2823 non-null   object 
 16  types          2823 non-null   object 
 17  num_listings   2823 non-null   int64  
 18  median_p

In [243]:
df_cleaned

,Name,id,average,avgweight,numwish,numowned,numplays,numfans,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,categories,types,num_listings,median_price
0,Gloomhaven,174430,8.79923,3.8549,15900.0,66360.0,332850.0,7003.0,2017.0,1.0,4.0,60.0,120.0,14.0,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[thematic, strategy]",52,120.000
1,Pandemic Legacy: Season 1,161936,8.61526,2.8400,10696.0,64193.0,228424.0,2779.0,2015.0,2.0,4.0,60.0,60.0,13.0,"[Environmental, Medical]","[thematic, strategy]",27,62.990
2,Brass: Birmingham,224517,8.66068,3.9170,9226.0,27062.0,43737.0,1459.0,2018.0,2.0,4.0,60.0,120.0,14.0,"[Economic, Industry / Manufacturing, Transport...",[strategy],15,120.000
3,Terraforming Mars,167791,8.43238,3.2388,17193.0,84252.0,349382.0,5550.0,2016.0,1.0,5.0,120.0,120.0,12.0,"[Economic, Environmental, Industry / Manufactu...",[strategy],29,48.990
4,Twilight Imperium: Fourth Edition,233078,8.69556,4.2224,7813.0,16328.0,18830.0,1582.0,2017.0,3.0,6.0,240.0,480.0,14.0,"[Civilization, Economic, Negotiation, Politica...","[thematic, strategy]",10,142.495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9913,Schollen Rollen,232437,6.01188,1.0000,25.0,239.0,712.0,41.0,2017.0,2.0,8.0,20.0,20.0,8.0,[Dice],[family],1,14.500
9939,Cornucopia,62343,5.83926,1.4444,66.0,529.0,527.0,70.0,2010.0,2.0,5.0,45.0,60.0,8.0,[Farming],[family],6,10.000
9951,Burgoo,31971,5.69934,1.2326,23.0,1890.0,1747.0,16.0,2014.0,2.0,5.0,15.0,15.0,10.0,[Abstract Strategy],[abstract],3,5.000
9983,"Congo Merc: The Congo, 1964",129307,6.28084,1.8462,52.0,321.0,165.0,80.0,2012.0,1.0,1.0,30.0,30.0,12.0,[Wargame],[war],1,9.000


In [260]:
def ohe_categories(df, cols_to_drop):
    """
    One-hot encodes game categories, then drops any columns in cols_to drop
    
    """
    mlb = MultiLabelBinarizer()
    df2 = df.join(pd.DataFrame(mlb.fit_transform(df.pop('categories')),
                          columns=mlb.classes_,
                          index=df.index))
    for col in cols_to_drop:
        if col in df2.columns:
            df2.drop(columns=[col], inplace=True)
    return df2

def ohe_types(df):
    """
    One-hot encodes game types, then drops the types that repeat with categories
    """
    mlb = MultiLabelBinarizer()
    df2 = df.join(pd.DataFrame(mlb.fit_transform(df.pop('types')),
                          columns=mlb.classes_,
                          index=df.index))
    for col in ['abstract', 'children\'s', 'party','war' ]:
        if col in df2.columns:
            df2.drop(columns=[col], inplace=True)
    return df2

In [202]:
ohe_categories(df_with_mp[:10],['abstract','war','party', ''])

,Name,id,average,avgweight,numwish,numowned,numplays,numfans,yearpublished,minplayers,...,Fighting,Industry / Manufacturing,Medical,Miniatures,Movies / TV / Radio theme,Negotiation,Political,Science Fiction,Space Exploration,Territory Building
0,Gloomhaven,174430,8.79923,3.8549,15900.0,66360.0,332850.0,7003.0,2017.0,1.0,...,1,0,0,1,0,0,0,0,0,0
1,Pandemic Legacy: Season 1,161936,8.61526,2.8400,10696.0,64193.0,228424.0,2779.0,2015.0,2.0,...,0,0,1,0,0,0,0,0,0,0
2,Brass: Birmingham,224517,8.66068,3.9170,9226.0,27062.0,43737.0,1459.0,2018.0,2.0,...,0,1,0,0,0,0,0,0,0,0
3,Terraforming Mars,167791,8.43238,3.2388,17193.0,84252.0,349382.0,5550.0,2016.0,1.0,...,0,1,0,0,0,0,0,1,1,1
4,Twilight Imperium: Fourth Edition,233078,8.69556,4.2224,7813.0,16328.0,18830.0,1582.0,2017.0,3.0,...,0,0,0,0,0,1,1,1,1,0
5,Through the Ages: A New Story of Civilization,182028,8.43130,4.4077,7725.0,26395.0,79438.0,1560.0,2015.0,2.0,...,0,0,0,0,0,0,0,0,0,0
6,Gaia Project,220308,8.49147,4.3410,8823.0,19473.0,52314.0,1328.0,2017.0,1.0,...,0,0,0,0,0,0,0,1,1,1
7,Star Wars: Rebellion,187645,8.42038,3.7054,10098.0,34130.0,33752.0,2129.0,2016.0,2.0,...,1,0,0,1,1,0,0,1,0,0
9,Great Western Trail,193738,8.29398,3.7049,9490.0,34737.0,91125.0,1497.0,2016.0,2.0,...,0,0,0,0,0,0,0,0,0,0
10,Gloomhaven: Jaws of the Lion,291457,8.92384,3.5134,4919.0,17513.0,32721.0,1084.0,2020.0,1.0,...,1,0,0,1,0,0,0,0,0,0


In [222]:
ohe_types(df_with_mp[:1000])['strategy'].value_counts()


1    537
0    463
Name: strategy, dtype: int64

In [203]:
category_dict.values()

dict_values(['Print & Play', 'Wargame', 'Abstract Strategy', 'Science Fiction', 'Card Game', 'Fantasy', 'Deduction', 'Horror', 'Economic', 'Civilization', 'Adventure', 'Party Game', 'Miniatures', 'World War II', 'Space Exploration', 'City Building', 'Ancient', 'Exploration', 'Medieval', 'Trains', 'Action / Dexterity', 'Pirates', 'Dice', 'Zombies', 'Murder/Mystery', 'American West', 'Mythology', 'Bluffing', "Children's Game", 'Animals', 'Nautical', 'Napoleonic', 'World War I', 'Political', 'Territory Building', 'Real-time', 'Negotiation', 'Modern Warfare', 'Racing', 'Puzzle', 'Industry / Manufacturing', 'Fighting', 'Environmental', 'Prehistoric', 'American Civil War', 'Word Game', 'Humor', 'Educational', 'Renaissance', 'Medical', 'Farming', 'Aviation / Flight', 'Sports', 'Religious', 'Age of Reason', 'Spies/Secret Agents', 'Game System', 'Video Game Theme', 'Mature / Adult', 'Comic Book / Strip', 'Pike and Shot', 'Vietnam War', 'Math', 'American Revolutionary War', 'Expansion for Base-g

In [214]:
ohe_categories(df_with_mp[:1000],[])['Party Game'].value_counts()

0    952
1     48
Name: Party Game, dtype: int64

In [217]:
list(category_dict.values())[:20]

['Print & Play',
 'Wargame',
 'Abstract Strategy',
 'Science Fiction',
 'Card Game',
 'Fantasy',
 'Deduction',
 'Horror',
 'Economic',
 'Civilization',
 'Adventure',
 'Party Game',
 'Miniatures',
 'World War II',
 'Space Exploration',
 'City Building',
 'Ancient',
 'Exploration',
 'Medieval',
 'Trains']

In [262]:
df3 = df_cleaned.copy()
df3 = ohe_categories(df3, list(category_dict.values())[11:] )
df3 = ohe_types(df3)
df3.reset_index(inplace=True, drop=True)
df3

,Name,id,average,baverage,avgweight,numwish,numowned,numplays,numfans,yearpublished,...,Economic,Fantasy,Horror,Print & Play,Science Fiction,Wargame,customizable,family,strategy,thematic
0,Gloomhaven,174430,8.79923,8.55546,3.8549,15900.0,66360.0,332850.0,7003.0,2017.0,...,0,1,0,0,0,0,0,0,1,1
1,Pandemic Legacy: Season 1,161936,8.61526,8.46581,2.8400,10696.0,64193.0,228424.0,2779.0,2015.0,...,0,0,0,0,0,0,0,0,1,1
2,Brass: Birmingham,224517,8.66068,8.36360,3.9170,9226.0,27062.0,43737.0,1459.0,2018.0,...,1,0,0,0,0,0,0,0,1,0
3,Terraforming Mars,167791,8.43238,8.28551,3.2388,17193.0,84252.0,349382.0,5550.0,2016.0,...,1,0,0,0,1,0,0,0,1,0
4,Twilight Imperium: Fourth Edition,233078,8.69556,8.24081,4.2224,7813.0,16328.0,18830.0,1582.0,2017.0,...,1,0,0,0,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,Schollen Rollen,232437,6.01188,5.54990,1.0000,25.0,239.0,712.0,41.0,2017.0,...,0,0,0,0,0,0,0,1,0,0
2819,Cornucopia,62343,5.83926,5.54949,1.4444,66.0,529.0,527.0,70.0,2010.0,...,0,0,0,0,0,0,0,1,0,0
2820,Burgoo,31971,5.69934,5.54931,1.2326,23.0,1890.0,1747.0,16.0,2014.0,...,0,0,0,0,0,0,0,0,0,0
2821,"Congo Merc: The Congo, 1964",129307,6.28084,5.54888,1.8462,52.0,321.0,165.0,80.0,2012.0,...,0,0,0,0,0,1,0,0,0,0


In [252]:
df3['plays/owner'] = df3['numplays']/df3['numowned']

In [253]:
df3

,index,Name,id,average,avgweight,numwish,numowned,numplays,numfans,yearpublished,...,Fantasy,Horror,Print & Play,Science Fiction,Wargame,customizable,family,strategy,thematic,plays/owner
0,0,Gloomhaven,174430,8.79923,3.8549,15900.0,66360.0,332850.0,7003.0,2017.0,...,1,0,0,0,0,0,0,1,1,5.015823
1,1,Pandemic Legacy: Season 1,161936,8.61526,2.8400,10696.0,64193.0,228424.0,2779.0,2015.0,...,0,0,0,0,0,0,0,1,1,3.558394
2,2,Brass: Birmingham,224517,8.66068,3.9170,9226.0,27062.0,43737.0,1459.0,2018.0,...,0,0,0,0,0,0,0,1,0,1.616178
3,3,Terraforming Mars,167791,8.43238,3.2388,17193.0,84252.0,349382.0,5550.0,2016.0,...,0,0,0,1,0,0,0,1,0,4.146869
4,4,Twilight Imperium: Fourth Edition,233078,8.69556,4.2224,7813.0,16328.0,18830.0,1582.0,2017.0,...,0,0,0,1,1,0,0,1,1,1.153234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,9913,Schollen Rollen,232437,6.01188,1.0000,25.0,239.0,712.0,41.0,2017.0,...,0,0,0,0,0,0,1,0,0,2.979079
2819,9939,Cornucopia,62343,5.83926,1.4444,66.0,529.0,527.0,70.0,2010.0,...,0,0,0,0,0,0,1,0,0,0.996219
2820,9951,Burgoo,31971,5.69934,1.2326,23.0,1890.0,1747.0,16.0,2014.0,...,0,0,0,0,0,0,0,0,0,0.924339
2821,9983,"Congo Merc: The Congo, 1964",129307,6.28084,1.8462,52.0,321.0,165.0,80.0,2012.0,...,0,0,0,0,1,0,0,0,0,0.514019


In [251]:
df3.iloc[2343]

index                                                5287
Name                 Plan Orange: Pacific War 1932 – 1935
id                                                 190232
average                                           7.55303
avgweight                                         3.41180
numwish                                          49.00000
numowned                                       1042.00000
numplays                                        143.00000
numfans                                          28.00000
yearpublished                                  2016.00000
minplayers                                        2.00000
maxplayers                                        2.00000
minplaytime                                     120.00000
maxplaytime                                     480.00000
minage                                           12.00000
num_listings                                            3
median_price                                     49.95000
Abstract Strat

In [264]:
np.log10(df3['numowned']).sort_values()

2737    2.071882
2806    2.120574
2746    2.152288
2807    2.161368
2625    2.173186
          ...   
3       4.925580
15      4.961269
210     4.962521
76      5.023483
50      5.044865
Name: numowned, Length: 2823, dtype: float64